# Importing Libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory as imp
from tensorflow.keras.losses import BinaryCrossentropy, categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Resizing, Rescaling, Dense, GlobalAveragePooling2D, BatchNormalization, RandomFlip, RandomContrast, RandomBrightness, InputLayer, Conv2D, Dropout, MaxPool2D, Flatten, MaxPooling2D
import matplotlib.pyplot as plt
import numpy as np
import math
import h5py

# Importing Data

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of data for validation
)

# Load training and validation data
train_data = train_datagen.flow_from_directory(
    r"data",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    r"data",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 8001 images belonging to 3 classes.
Found 1999 images belonging to 3 classes.


In [4]:
steps_per_epoch = math.ceil(train_data.samples / train_data.batch_size)
validation_steps = math.ceil(val_data.samples / val_data.batch_size)

In [10]:
CONFIGURATIONS = {"Class_Names": ["Angry", "Happy", "Sad"],
                  "IM_SIZE" : 224,}
                  

# Data Visualisation

In [ ]:
plt.figure(figsize = (6,6))

for images, labels in training_data:
    for i in range(16):
        ax = plt.subplot(4,4, i+1)
        plt.imshow(images[i]/255)
        plt.title(CONFIGURATIONS["Class_Names"][tf.argmax(labels[i], axis = 0).numpy()])
        plt.axis("off")

# Data Preprocessing

In [5]:
def convert_to_tf_dataset(iterator):
    dataset = tf.data.Dataset.from_generator(
        lambda: iterator,
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, iterator.num_classes), dtype=tf.float32)
        )
    )
    return dataset

train_dataset = convert_to_tf_dataset(train_data)
val_dataset = convert_to_tf_dataset(val_data)

In [6]:
training_data = (train_dataset
                .prefetch(tf.data.AUTOTUNE))
val_data = (val_dataset
                .prefetch(tf.data.AUTOTUNE))

# Model

In [7]:
early_stopping = EarlyStopping(monitor='val_loss',
                                patience=5,           
                                verbose=1,            
                                restore_best_weights=True,)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
backbone = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top = False,
    weights='imagenet',
    input_shape=(CONFIGURATIONS["IM_SIZE"], CONFIGURATIONS["IM_SIZE"], 3),
    )

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [12]:
backbone.trainable = True

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape= (None, None, 3),),
    resize_rescale_layer,
    Conv2D(filters = 64, kernel_size = 3, strides=1, padding='valid', activation='relu',),
    BatchNormalization(),
    MaxPool2D(pool_size = 3, strides = 1,),
    Dropout(rate = 0.2,),
    Conv2D(filters = 32, kernel_size = 3, strides=1, padding='valid', activation='relu',),
    BatchNormalization(),
    MaxPool2D(pool_size = 3, strides = 2,),

    Flatten(),

    Dense(128, activation = 'relu', kernel_regularizer=L2(0.01)),
    BatchNormalization(),
    Dropout(rate = 0.2,),

    Dense(64, activation = 'relu', kernel_regularizer=L2(0.01)),
    BatchNormalization(),

    Dense(6, activation = 'softmax',)
])
model.summary()

In [13]:
input = tf.keras.layers.Input(shape=(None, None, 3))

x = backbone(input, training = False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = BatchNormalization()(x)
x = Dense(64, activation = "relu")(x)
output = Dense(6, activation = "softmax")(x)

finetuned_model = Model(input, output)

In [14]:
pretrained_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, None, 3)),
    #resize_rescale_layer,
    backbone,
    GlobalAveragePooling2D(),
    Dense(128, activation = "relu", kernel_regularizer= L2),
    BatchNormalization(),
    Dense(68, activation = "relu", kernel_regularizer= L2),
    Dense(3, activation = "softmax"),

    ])
pretrained_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, None, None, 1280)    │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 68)                  │           8,772 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             207 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,431,443 (9.28 MB)

 Trainable params: 2,397,075 (9.14 MB)

 Non-trainable params: 34,368 (134.25 KB)

In [15]:
loss = ['categorical_crossentropy']
metrics = ["accuracy"]

In [16]:
pretrained_model.compile(optimizer=Adam(learning_rate=0.01),
             loss='categorical_crossentropy',
             metrics=metrics)

In [17]:
METRIC_DIR = './logs/metrics'
train_writer = tf.summary.create_file_writer(METRIC_DIR)
LOG_DIR = './logs/'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq = 1,)

In [18]:
history = pretrained_model.fit(training_data, epochs = 100, validation_data = val_data, steps_per_epoch=steps_per_epoch,
    callbacks=[tensorboard_callback, early_stopping], validation_steps = validation_steps)

Epoch 1/100
127/251 ━━━━━━━━━━━━━━━━━━━━ 7:02 3s/step - accuracy: 0.4172 - loss: 3.1439

KeyboardInterrupt: 

# Computer Vision Part

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Initialize the video capture with the default camera (index 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video capture")
else:
    # Initialize Mediapipe Face Detection
    mpFaceDetection = mp.solutions.face_detection
    mpDraw = mp.solutions.drawing_utils
    faceDetection = mpFaceDetection.FaceDetection(0.75)

   

    CONFIGURATIONS = {
        "Class_Names": ['Anger', 'Happy', 'Sad'],  # Replace with actual class names
        "IM_SIZE": (224, 224)  # Ensure this is a tuple (width, height)
    }

    while True:
        # Capture frame-by-frame
        success, frame = cap.read()
        if not success:
            print("Error: Failed to capture image")
            break

        frame = cv2.flip(frame, 1)
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = faceDetection.process(imgRGB)

        # Draw face detection boxes and make predictions on each detected face
        if results.detections:
            for index, detection in enumerate(results.detections):
                mpDraw.draw_detection(frame, detection)
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                bbox = (
                    int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                    int(bboxC.width * iw), int(bboxC.height * ih)
                )
                x, y, w, h = bbox
                bbox_region = frame[y:y+h+20, x:x+w+20]

                if bbox_region.size == 0:
                    continue  # Skip if bbox_region is empty

                # Resize the face region to the input size expected by your model
                img = cv2.resize(bbox_region, CONFIGURATIONS["IM_SIZE"])

                # Convert BGR to RGB
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Normalize the image
                img = img / 255.0
                
                # Expand dimensions to match the model input shape
                img = np.expand_dims(img, axis=0)

                # Make predictions
                predictions = pretrained_model.predict(img)
                predicted_class = np.argmax(predictions, axis=1)
                name = CONFIGURATIONS["Class_Names"][predicted_class[0]]

                # Draw the bounding box and label on the original image
                cv2.putText(frame, f'{name}', (bbox[0], bbox[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)

        # Display the resulting frame
        cv2.imshow('Webcam Feed', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
